# Converting .xml to pd.DataFrame

In [1]:
#Import der packages
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

In [33]:
#Import von .xml Daten

import xml.etree.ElementTree as ET


def read_xml(path):
   xml = ET.parse(path).getroot()
   return xml

def search_for_name(tree, attr):
   names = []
   for child in tree:
       if child.tag == attr:
           names += [child.text]
       else:
           names += search_for_name(child, attr)
   return names

#Test
if __name__=="__main__":
   xml = read_xml("MDB_STAMMDATEN.XML")
   names = search_for_name(xml,"GESCHLECHT")
   
print(names[:4])
print(len(names))

['männlich', 'männlich', 'weiblich', 'weiblich']
4073


In [35]:
#Erste Ansätze für ein DataFrame mit den folgenden Spalten

#Merkmale
liste_merk = ["ID", "NACHNAME", "VORNAME","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND", "WP"]

#Einmaliges zusammensetzten einer Liste von Listen mit den Ausprägungen der obigen Merkmale
column_list = []
for attr in liste_merk:
    if __name__=="__main__":
        xml = read_xml("MDB_STAMMDATEN.XML")
        names = search_for_name(xml, attr)
        column_list.append(names)

In [36]:
#Nun wird die Kompatibilität der einzelnen Listen überprüft. Es soll ein pd.DataFrame erstellt werden und dafür müssen die 
#Spaltenlängen übereinstimmen

print(len(column_list))

for i in column_list:
    print(len(i))

9
4073
4336
4336
4073
4073
4073
4073
4073
11600


In [10]:
#Mit diesen verkürzten Liste lässt sich schon ein DataFrame erstellen
liste_merk_2 = ["ID","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND"]

In [13]:
#Etwas umstänlich, aber wirksam wird ein DF zusammen gestellt und ein erster Mangel behoben 
#(none values werden durch "Deutschland erszetz")

df_first = pd.DataFrame(np.column_stack([column_list[0],column_list[3],column_list[4],column_list[5],column_list[6],column_list[7]]), columns = liste_merk_2)

df_first.loc[df_first["GEBURTSLAND"].isnull(), "GEBURTSLAND"] = "Deutschland"


print(df_first.head())

   

         ID GESCHLECHT PARTEI_KURZ GEBURTSDATUM  \
0  11000001   männlich         CDU         1930   
1  11000002   männlich         FDP         1909   
2  11000003   weiblich         CDU         1913   
3  11000004   weiblich         CDU         1933   
4  11000005   männlich         CDU         1950   

                                               BERUF  GEBURTSLAND  
0  Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...  Deutschland  
1                             Rechtsanwalt und Notar  Deutschland  
2                                    Hilfsreferentin  Jugoslawien  
3                                             Ärztin  Deutschland  
4                      Mathematiker, Geschäftsführer  Deutschland  


In [12]:
#hier besteht noch Verbesserungsbedarf, dazu aber später mehr
df_first.dtypes

ID              object
GESCHLECHT      object
PARTEI_KURZ     object
GEBURTSDATUM    object
BERUF           object
GEBURTSLAND     object
dtype: object

In [21]:
#Mit Hilfe einer höheren Macht lässt sich der Parsing-Prozess weiterentwickeln
#Hier für die Vor- und Nachnamen

def search_for_nameRoots(root):
 names = []
 for child in root:
     if child.tag == "NAMEN":
         names += [child]
     else:
         names += search_for_nameRoots(child)
 return names

xml = read_xml(path = "MDB_STAMMDATEN.XML")

nameRoots = search_for_nameRoots(xml)

firstNameElements = []
for element in nameRoots:
   firstNameElements += [element[0].find('VORNAME').text]

lastNameElements = []
for element in nameRoots:
   lastNameElements += [element[0].find('NACHNAME').text]

df_first["NACHNAME"] = lastNameElements
df_first["VORNAME"] = firstNameElements


In [22]:
#Die Integration ist geglückt
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich


In [37]:
#Nun muss der Parsing-Prozess nochmal umgestaltet werden, um zu einer spezifischen ID/einem spezifischen MdB die Nummern
#aller Wahlperioden zu finden.
#Langfrisitg soll auf diese Perioden umindizoiert werden.


#Die folgende Funktion soll eine Liste von Listen dieser nummerierten Wahlperioden produzieren

def search_for_wahlRoots(root):
    
    #nestet function (Entscheidend ist hier der Break beim nächsten "ID" tag)
    def search_for_wpRoots(root2):
        wplist2 = []
        for child2 in root2:
            if child2.tag == "WP":
                wplist2 += [child2]
            elif child2.tag == "ID":
                break
            else:
                wplist2 += search_for_wpRoots(child2)
        return nwplist2
    #ende nested function
    
    wplist = []
    for child in root:
        if child.tag == "WAHLPERIODEN":
            wplist += [search_for_nameRoots(child)]
        else:
            wplist += search_for_wahlRoots(child)
    return wplist


xml = read_xml(path = "MDB_STAMMDATEN.XML")

WProots = search_for_nameRoots1(xml)
    
#Nunn müssen noch die gewünschten Nummern aus dem Code extrahiert werden    

def extracting_WP(list1):
    WPelements = []
    for i in list1:
        sub_list = []
        for a in i:
            sub_list += [a.text]
        WPelements += [sub_list]
    return WPelements

WPs = extracting_WP(WProots)


#Testen der neuen Liste auf längenmäßige Kompatibilität mit dem DF  
print(WPs[:4])
print(len(WPs))

[['5', '6', '7', '8', '9', '10', '11'], ['3', '4', '5', '6', '7'], ['2', '3', '4'], ['11', '12']]
4073


In [39]:
#Erweiterung
df_first["WAHLPERIODEN"] = WPs

df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred,"[5, 6, 7, 8, 9, 10, 11]"
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst,"[3, 4, 5, 6, 7]"
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie,"[2, 3, 4]"
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else,"[11, 12]"
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich,"[15, 16, 12, 13, 14]"
